In [351]:
import pandas as pd
import numpy as np

In [352]:
df = pd.read_csv('c:\\Users\\bkser\\Downloads\\python\\base_doencas_cardiacas.csv')

## Escolaridade

D00901 – curso mais elevado frequentado

D014 – se concluiu esse curso

D00201 – se frequenta escola atualmente

D008 – se já frequentou escola

In [353]:
# Garantir que os atributos são numéricos
df['D00901'] = pd.to_numeric(df['D00901'], errors='coerce')
df['D014'] = pd.to_numeric(df['D014'], errors='coerce')
df['D00201'] = pd.to_numeric(df['D00201'], errors='coerce')
df['D008'] = pd.to_numeric(df['D008'], errors='coerce')

# 1. Corrigir D014 para infantil (Creche ou Pré-escola) → assumir como concluído se estiver nan quando D00901 = 1 e 2
cond = df['D00901'].isin([1, 2]) & df['D014'].isna()
df.loc[cond, 'D014'] = 1

# 2. Função para mapear escolaridade final (0 a 5)
def map_escolaridade(row):
    curso = row['D00901']
    concluiu = row['D014']
    freq_atualmente = row['D00201']
    freq_antes = row['D008']

    if pd.isna(curso):
        # Se não tem informação de curso, mas também nunca frequentou
        if freq_atualmente != 1 and freq_antes != 1:
            return 0  # Nunca frequentou
        else:
            return np.nan

    # Mapeamento baseado no questionário da PNS 2019
    if curso in [1, 2]:            # Creche, Pré-escola
        categoria = 1
    elif curso in [3, 4, 5, 6, 7, 8]:  # Alfabetização, primário, EJA fundamental etc.
        categoria = 2
    elif curso in [9, 10, 11]:     # Científico, médio, EJA médio
        categoria = 3
    elif curso == 12:              # Superior - graduação
        categoria = 4
    elif curso in [13, 14, 15]:    # Pós-graduação
        categoria = 5
    else:
        return np.nan

    # Rebaixar se não concluiu
    if concluiu == 2:
        categoria -= 1
        if categoria < 0:
            categoria = 0

    return categoria

# Aplicar função
df['Escolaridade'] = df.apply(map_escolaridade, axis=1)

# Remover colunas intermediárias
df.drop(columns=['D00201', 'D008', 'D00901', 'D014'], inplace=True)

# Verificar resultado
print(df['Escolaridade'].value_counts(dropna=False))

# 0: Nunca frequentou escola
# 1: Educação infantil
# 2: Ensino fundamental
# 3: Ensino médio
# 4: Ensino superior
# 5: Pós-graduação


Escolaridade
1.0    9187
3.0    4939
2.0    4072
0.0    2767
4.0    1800
5.0     940
NaN     687
Name: count, dtype: int64


## Renda 

E011: número de trabalhos na semana

E014011: se era trabalhador não remunerado ajudando parente/domicílio

E01601: se recebia dinheiro no trabalho principal

E01602: valor da renda do trabalho principal

E01801: se recebia dinheiro em outro(s) trabalho(s)

E01802: valor da renda dos outros trabalhos


Regras:

1- Se E01601 != 1 ou E01801 != 1, a renda respectiva é 0;

2- Se a pessoa é não remunerada (E014011 = 1, 2, 3), renda é 0;

3- Se E01602 ou E01802 estiverem NaN, mas a pessoa disse que recebia (E01601 ou E01801 = 1), considerar como NaN final.

4- Renda total é a soma de E01602 e E01802 (ajustadas com base nas respostas anteriores).

5- A variável final será chamada Renda e os outros atributos serão removidos.

In [354]:
# Garantir que os campos estão numéricos
for col in ['E011', 'E014011', 'E01601', 'E01602', 'E01801', 'E01802']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Função para calcular a renda final
def calcular_renda(row):
    # Regra 2: trabalhador não remunerado (1, 2 ou 3)
    if row['E014011'] in [1, 2, 3]:
        return 0.0

    # Regra 1 + 3: verificar se recebia dinheiro
    if row['E01601'] == 1:
        renda1 = row['E01602'] if pd.notna(row['E01602']) else np.nan
    else:
        renda1 = 0.0

    if row['E01801'] == 1:
        renda2 = row['E01802'] if pd.notna(row['E01802']) else np.nan
    else:
        renda2 = 0.0

    # Regra 4: soma ajustada
    if pd.isna(renda1) or pd.isna(renda2):
        if pd.notna(renda1) and pd.isna(renda2):
            return renda1
        elif pd.notna(renda2) and pd.isna(renda1):
            return renda2
        else:
            return np.nan

    return renda1 + renda2

# Aplicar a função
df['Renda'] = df.apply(calcular_renda, axis=1)

# Remover colunas usadas
df.drop(columns=['E014011', 'E01601', 'E01602', 'E01801', 'E01802'], inplace=True)

# Verificar
print(df['Renda'].isna().sum())

0


In [355]:
print((df['Renda'] == 0.0).sum())

12249


In [356]:
df[df['Renda'] == 0.0][['Renda']]

,Renda
0,0.0
4,0.0
6,0.0
9,0.0
10,0.0
...,...
24355,0.0
24366,0.0
24367,0.0
24375,0.0


## Horas trabalhadas

E017 - 1 trabalho
E019 - outros trabalhos


In [357]:
# Certifique-se de que os campos são numéricos
df['E017'] = pd.to_numeric(df['E017'], errors='coerce')
df['E019'] = pd.to_numeric(df['E019'], errors='coerce')

# Criar a variável final
df['Horas_trabalhadas'] = df['E017'].fillna(0) + df['E019'].fillna(0)

# Se quiser que NaN em ambos resulte em NaN no final, use:
# df['Horas_trabalhadas'] = df[['E017', 'E019']].sum(axis=1, min_count=1)

# Remover colunas antigas
df.drop(columns=['E017', 'E019'], inplace=True)

# Verificar resultado
print(df['Horas_trabalhadas'].describe())
print(df['Horas_trabalhadas'].isna().sum())

count    24392.000000
mean        18.881437
std         21.333164
min          0.000000
25%          0.000000
50%          5.000000
75%         40.000000
max        120.000000
Name: Horas_trabalhadas, dtype: float64
0


In [358]:
df[(df['Renda'] == 0.0) & (df['Horas_trabalhadas'] == 0.0)][['Renda', 'Horas_trabalhadas']]

,Renda,Horas_trabalhadas
0,0.0,0.0
4,0.0,0.0
6,0.0,0.0
9,0.0,0.0
10,0.0,0.0
...,...,...
24355,0.0,0.0
24366,0.0,0.0
24367,0.0,0.0
24375,0.0,0.0


## Plano de saúde 

I00102 - se tem plano
I006 - avaliação do plano

0     | Não tem plano                
1     | Avaliação muito boa ou boa   
2     | Avaliação regular            
3     | Avaliação ruim ou muito ruim 
4     | Nunca usou o plano   

In [359]:
def classificar_plano(row):
    tem_plano = row['I00102']
    avaliacao = row['I006']
    
    if tem_plano == 2:
        return 0.0  # Não tem plano
    elif avaliacao in [1, 2]:
        return 1.0  # Muito bom ou bom
    elif avaliacao == 3:
        return 2.0  # Regular
    elif avaliacao in [4, 5]:
        return 3.0  # Ruim ou muito ruim
    elif avaliacao == 6:
        return 4.0  # Nunca usou
    else:
        return np.nan  # Valores faltantes ou inválidos

df['Avaliacao_plano'] = df.apply(classificar_plano, axis=1)

df = df.drop(columns=['I006'])

print(df['Avaliacao_plano'].value_counts(dropna=False))
print(df['Avaliacao_plano'].isna().sum())

Avaliacao_plano
0.0    19784
1.0     3414
2.0      941
3.0      157
4.0       96
Name: count, dtype: int64
0


I004 - se é de instituição pública

I00103 - se é morador 

C00301 == I001031 - se é titular 


0     | Não tem plano e/ou não é morador + titular do plano              
1     | Plano é de instituição pública   
2     | Plano não é de instituição pública      

In [360]:
def definir_plano_publico(row):
    if row['I00102'] != 1:
        return 0.0  # Não tem plano
    if row['I00103'] != 1 and row['C00301'] != row['I001031']: # não é morador
        return 0.0  # não é titular do plano
    if row['I004'] == 1:
        return 1.0  # Plano é de instituição pública
    if row['I004'] == 2:
        return 2.0  # Plano não é de instituição pública
    
    return 0.0

# Aplicar a função
df['Plano_publico'] = df.apply(definir_plano_publico, axis=1)

# Remover colunas utilizadas
df = df.drop(columns=['I004'])

print(df['Plano_publico'].value_counts(dropna=False))
print(df['Plano_publico'].isna().sum())

Plano_publico
0.0    20907
2.0     2158
1.0     1327
Name: count, dtype: int64
0


In [361]:
def Cobertura_plano(row):
    # Se não tem plano, retorna 0.0
    if pd.isna(row['I00102']) or row['I00102'] != 1:
        return 0.0

    # Verifica titularidade
    if pd.isna(row['I00103']) or row['I00103'] != 1:
        return 0.0
    if pd.isna(row['C00301']) or pd.isna(row['I001031']) or row['C00301'] != row['I001031']:
        return 0.0

    # Verifica cobertura
    coberturas = [
        row['I00401'] == 1 if pd.notna(row['I00401']) else False,
        row['I00402'] == 1 if pd.notna(row['I00402']) else False,
        row['I00403'] == 1 if pd.notna(row['I00403']) else False
    ]
    total_cobertura = sum(coberturas)

    if total_cobertura == 1:
        return 1.0
    elif total_cobertura == 2:
        return 2.0
    elif total_cobertura == 3:
        return 3.0
    elif total_cobertura == 0:
        return 4.0  # Não cobre nada, apesar de ter plano e titular
    else:
        return np.nan  # Caso raro de erro

df['Cobertura_plano'] = df.apply(Cobertura_plano, axis=1)

# Agora sim pode excluir os atributos
df = df.drop(columns=['I00102', 'I00401', 'I00402', 'I00403', 'I00103', 'I001031'])

# Visualização
print(df['Cobertura_plano'].value_counts(dropna=False))
print("Valores nulos:", df['Cobertura_plano'].isna().sum())


Cobertura_plano
0.0    20907
3.0     3253
2.0      177
1.0       37
4.0       18
Name: count, dtype: int64
Valores nulos: 0


## Mobilidade 

In [362]:
def classificar_mobilidade(row):
    usa_aparelho = row['G059'] == 1
    cadeira_rodas = row['G060'] == 1
    bengala_muleta = row['G062'] == 1

    if not usa_aparelho:
        return 0.0  # Não usa nenhum aparelho
    if cadeira_rodas:
        return 1.0  # Só cadeira de rodas
    if bengala_muleta:
        return 2.0  # Só bengala/muleta/andador
    if cadeira_rodas and bengala_muleta:
        return 3.0  # Usa os dois
    if not cadeira_rodas and not bengala_muleta:
        return 4.0  # Não usa nenhum dos dois  
    return np.nan  # Caso de inconsistência ou valores faltantes

df['Mobilidade_auxiliar'] = df.apply(classificar_mobilidade, axis=1)

# Remover colunas antigas, se quiser:
df = df.drop(columns=['G059', 'G060', 'G062'])

# Visualizar resultado:
print(df['Mobilidade_auxiliar'].value_counts(dropna=False))
print(df['Mobilidade_auxiliar'].isna().sum())

Mobilidade_auxiliar
0.0    23656
2.0      527
1.0      154
4.0       55
Name: count, dtype: int64
0


## Exposição no trabalho

In [363]:
def marcar_exposicao_quimica(row):
    trabalho = row['E011']
    exposicao = row['M011011']
    
    if trabalho in [1, 2, 3]:  # Pessoa trabalha
        if exposicao == 1:
            return 1  # Exposto
        elif exposicao == 2:
            return 2  # Não exposto
        else:
            return np.nan  # Outro valor ou faltante
    else:
        return 2  # Pessoa não trabalha → marcar como "não exposto"

df['Exposicao_quimica'] = df.apply(marcar_exposicao_quimica, axis=1)

df = df.drop(columns=['M011011'])

# Visualizar resultado:
print(df['Exposicao_quimica'].value_counts(dropna=False))
print(df['Exposicao_quimica'].isna().sum())

Exposicao_quimica
2    22975
1     1417
Name: count, dtype: int64
0


In [364]:
def marcar_exposicao_ruido(row):
    trabalho = row['E011']
    exposicao = row['M011021']
    
    if trabalho in [1, 2, 3]:  # Pessoa trabalha
        if exposicao == 1:
            return 1  # Exposto
        elif exposicao == 2:
            return 2  # Não exposto
        else:
            return np.nan  # Outro valor ou faltante
    else:
        return 2  # Pessoa não trabalha → marcar como "não exposto"

df['Exposicao_ruido'] = df.apply(marcar_exposicao_ruido, axis=1)

df = df.drop(columns=['M011021'])

# Visualizar resultado:
print(df['Exposicao_ruido'].value_counts(dropna=False))
print(df['Exposicao_ruido'].isna().sum())

Exposicao_ruido
2    22130
1     2262
Name: count, dtype: int64
0


In [365]:
def marcar_exposicao_lixo(row):
    trabalho = row['E011']
    exposicao = row['M011051']
    
    if trabalho in [1, 2, 3]:  # Pessoa trabalha
        if exposicao == 1:
            return 1  # Exposto
        elif exposicao == 2:
            return 2  # Não exposto
        else:
            return np.nan  # Outro valor ou faltante
    else:
        return 2  # Pessoa não trabalha → marcar como "não exposto"

df['Exposicao_lixo'] = df.apply(marcar_exposicao_lixo, axis=1)

df = df.drop(columns=['M011051'])

# Visualizar resultado:
print(df['Exposicao_lixo'].value_counts(dropna=False))
print(df['Exposicao_lixo'].isna().sum())

Exposicao_lixo
2    23612
1      780
Name: count, dtype: int64
0


In [366]:
def marcar_exposicao_minerio(row):
    trabalho = row['E011']
    exposicao = row['M011071']
    
    if trabalho in [1, 2, 3]:  # Pessoa trabalha
        if exposicao == 1:
            return 1  # Exposto
        elif exposicao == 2:
            return 2  # Não exposto
        else:
            return np.nan  # Outro valor ou faltante
    else:
        return 2  # Pessoa não trabalha → marcar como "não exposto"

df['Exposicao_minerio'] = df.apply(marcar_exposicao_minerio, axis=1)

df = df.drop(columns=['M011071'])

# Visualizar resultado:
print(df['Exposicao_minerio'].value_counts(dropna=False))
print(df['Exposicao_minerio'].isna().sum())

Exposicao_minerio
2    23026
1     1366
Name: count, dtype: int64
0


In [367]:
# df.drop(columns=['E011',

In [368]:
df.to_csv('c:\\Users\\bkser\\Downloads\\python\\base_tratada.csv', index=False)

In [369]:
df.columns[df.isnull().any()]

Index(['D01301', 'D01303', 'E002', 'E003', 'E004', 'E005', 'E006011', 'E008',
       'E010010', 'E011', 'E01603', 'E01605', 'E01604', 'E01804', 'I012',
       'J00402', 'J00404', 'J00801', 'M00203', 'M00302', 'N004', 'N005',
       'N008', 'P00103', 'P00104', 'P00403', 'P00404', 'P01001', 'P02801',
       'P029', 'P035', 'P03701', 'P03702', 'P051', 'P052', 'P053', 'P05401',
       'P05404', 'P05407', 'P05410', 'P05413', 'P05416', 'P05419', 'Q00201',
       'Q03001', 'Q060', 'Q06307', 'Q06308', 'Q06309', 'Q06310', 'Q06311',
       'Q064', 'Q06506', 'Q06507', 'Q06508', 'Q06509', 'Q06601', 'Q067',
       'R010', 'R025', 'R028', 'R029', 'R030', 'R031', 'R032', 'R033', 'R034',
       'R03601', 'R03607', 'R03608', 'R03610', 'S065', 'H002', 'H003', 'H004',
       'H010', 'Escolaridade'],
      dtype='object')

In [370]:
(df.isnull().sum() > 0).sum()

np.int64(77)